In [6]:
import librosa
import numpy as np
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf 
import tensorflow_io as tfio
from sklearn.model_selection import train_test_split 
from keras.layers import Dense, Dropout, LSTM 

Extract MFCCs

In [7]:
def extractFeatures(filePath): 
    audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast')  
    mfccs = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40) 
    mfccsProcessed = np.mean(mfccs.T,axis=0) 
    return mfccsProcessed 


Loading and labelling data

In [8]:

def loadAudioFiles(filePaths, label):
    features = []
    for file in filePaths:
        mfccsProcessed = extractFeatures(file)
        features.append(mfccsProcessed)
    return np.array(features), np.array([label] * len(features))

Importing files

In [9]:
correctFiles = ['audioFiles/adamKhaa .wav','audioFiles/dadKhaa.wav','audioFiles/hennaKhaa.wav','audioFiles/meKhaa.wav']
incorrectFiles = ['audioFiles/wrongKhaa1.wav','audioFiles/wrongKhaa2.wav','audioFiles/wrongKhaa3.wav']

In [10]:
correctFeatures, correctLabels = loadAudioFiles(correctFiles, 1)
incorrectFeatures, incorrectlabels = loadAudioFiles(incorrectFiles, 0)